<a href="https://colab.research.google.com/github/ThiagoPereira83/Machine_learning/blob/main/transfer_learning_em_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Importação das bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds

# Versão do TensorFlow
print(tf.__version__)



2.15.0


In [7]:
# Dataset "Cats vs Dogs"
(ds_train, ds_test), ds_info = tfds.load('cats_vs_dogs', split=['train[:80%]', 'train[80%:]'], with_info=True, as_supervised=True)

# Redimensionar as imagens
def preprocess(image, label):
    image = tf.image.resize(image, (160, 160))
    image = image / 255.0
    return image, label

# Preprocessamento ao dataset
ds_train = ds_train.map(preprocess).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteUHTE08/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [9]:
# Modelo MobileNetV2 pré-treinado sem a camada superior
base_model = MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')

# Congelar o modelo base
base_model.trainable = False

# Adicionar camadas superiores
model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Arquitetura do modelo
model.summary()



9406464/9406464 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 2259265 (8.62 MB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 2257984 (8.61 MB)
__________________

In [ ]:
# Treinar o modelo
history = model.fit(ds_train, epochs=10, validation_data=ds_test)

# Avaliar o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(ds_test)
print(f'Test accuracy: {test_accuracy}')



Epoch 1/10
582/582 [==============================] - 484s 826ms/step - loss: 0.1073 - accuracy: 0.9569 - val_loss: 0.0609 - val_accuracy: 0.9781
Epoch 2/10
246/582 [===========>..................] - ETA: 3:14 - loss: 0.0679 - accuracy: 0.9724